## Geofencing ligne

In [2]:
import cv2
import numpy as np

# Définir la ligne de geofencing
LINE_Y = 300  # Position verticale de la ligne
LINE_COLOR = (0, 255, 0)
LINE_THICKNESS = 2

# Capture vidéo (0 pour la webcam locale, ou l'URL de la caméra IP)
cap = cv2.VideoCapture(0)

# Initialisation du soustracteur d'arrière-plan
bg_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

def detect_crossing(y, h, line_y):
    """Détecte si le rectangle franchit la ligne virtuelle."""
    return y <= line_y <= (y + h)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prétraitement de l'image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Appliquer le soustracteur d'arrière-plan
    mask = bg_subtractor.apply(blurred)
    mask = cv2.threshold(mask, 25, 255, cv2.THRESH_BINARY)[1]

    # Trouver les contours des objets détectés
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Tracer la ligne de geofencing
    cv2.line(frame, (0, LINE_Y), (frame.shape[1], LINE_Y), LINE_COLOR, LINE_THICKNESS)

    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Filtrer les petits objets
            x, y, w, h = cv2.boundingRect(contour)  # Extraire les coordonnées du rectangle
            if detect_crossing(y, h, LINE_Y):
                cv2.putText(frame, "Franchissement detecte!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # Affichage des images
    cv2.imshow("Video", frame)
    cv2.imshow("Masque", mask)

    # Quitter avec la touche 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Geofencing polygone

In [ ]:
import cv2
import numpy as np

# Définir les points du polygone (ajustez selon votre zone d'intérêt)
polygon_points = np.array([[100, 200], [400, 200], [450, 300], [150, 350]], np.int32)
polygon_points = polygon_points.reshape((-1, 1, 2))

# Capture vidéo (0 pour webcam locale ou URL pour caméra IP)
cap = cv2.VideoCapture(0)

# Initialisation du soustracteur d'arrière-plan
bg_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

def is_inside_polygon(x, y, w, h, polygon):
    """Vérifie si un rectangle entre dans un polygone."""
    rect_center = (x + w // 2, y + h // 2)
    return cv2.pointPolygonTest(polygon, rect_center, False) >= 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prétraitement
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Appliquer le soustracteur d'arrière-plan
    mask = bg_subtractor.apply(blurred)
    mask = cv2.threshold(mask, 25, 255, cv2.THRESH_BINARY)[1]

    # Trouver les contours des objets détectés
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Dessiner le polygone sur le frame
    cv2.polylines(frame, [polygon_points], isClosed=True, color=(0, 255, 0), thickness=3)

    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Filtrer les petits objets
            x, y, w, h = cv2.boundingRect(contour)
            if is_inside_polygon(x, y, w, h, polygon_points):
                cv2.putText(frame, "Intrusion detectee!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # Affichage
    cv2.imshow("Video", frame)
    cv2.imshow("Masque", mask)

    # Quitter avec la touche 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## limite FPS and size

In [5]:
import cv2
import numpy as np

# Définir les points du polygone (à ajuster)
polygon_points = np.array([[50, 100], [200, 100], [450, 300], [150, 350]], np.int32)
polygon_points = polygon_points.reshape((-1, 1, 2))

# Capture vidéo (0 pour webcam locale ou URL pour caméra IP)
cap = cv2.VideoCapture(0)

# Réduire la taille de la vidéo
FRAME_WIDTH = 640
FRAME_HEIGHT = 360

# Limiter les FPS (attente entre chaque frame en millisecondes)
TARGET_FPS = 10
frame_delay = int(1000 / TARGET_FPS)  # Conversion FPS -> millisecondes

# Initialisation du soustracteur d'arrière-plan
bg_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

def is_inside_polygon(x, y, w, h, polygon):
    """Vérifie si un rectangle entre dans un polygone."""
    rect_center = (x + w // 2, y + h // 2)
    return cv2.pointPolygonTest(polygon, rect_center, False) >= 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Redimensionner le frame
    frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))

    # Prétraitement
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Appliquer le soustracteur d'arrière-plan
    mask = bg_subtractor.apply(blurred)
    mask = cv2.threshold(mask, 25, 255, cv2.THRESH_BINARY)[1]

    # Trouver les contours des objets détectés
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Dessiner le polygone sur le frame
    cv2.polylines(frame, [polygon_points], isClosed=True, color=(0, 255, 0), thickness=3)

    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Filtrer les petits objets
            x, y, w, h = cv2.boundingRect(contour)
            if is_inside_polygon(x, y, w, h, polygon_points):
                cv2.putText(frame, "Intrusion detectee!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # Affichage
    cv2.imshow("Video", frame)
    cv2.imshow("Masque", mask)

    # Quitter avec la touche 'q'
    if cv2.waitKey(frame_delay) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()